In [1]:
query = """
{
  governances {
    totalCount
    edges {
      node {
        # platform
        platform_id: id
        name: name
        # website
        # additional
        propositionStatusMetrics {
          proposals_count: totalCount
        }
        # votes_count
      }
    }
  }
}

# * `platform`
# * `platform_id`
# * `name` (may be nan)
# * `website` (may be nan)
# * `additional` (may contain website, social, etc. info, may be nan)
# * `votes_count` (the number of unique votes in the deployment)
#   - snapshot reports the sum of the votes in the deployment
#   - for daostack, aragon, and dao haus we report the total number of votes, including duplicates by the same voter
# * `proposals_count` (the number of proposals in the deployment)
#   - used Realms JS SDK
"""

In [2]:
# hit gql https://graphql.messari.io/query
import requests

response = requests.post('https://graphql.messari.io/query', json={'query': query})
print(response.status_code)
data = response.json()
len(data['data']['governances']['edges'])

200


885

In [3]:
import pandas as pd
# df from edges
df = pd.DataFrame(data['data']['governances']['edges'])
# expand the node column
df = pd.concat([df.drop(['node'], axis=1), df['node'].apply(pd.Series)], axis=1)
df.head()

,platform_id,name,propositionStatusMetrics
0,00233765-eb17-4e75-954f-82258bb990aa,Shell Protocol,{'proposals_count': 0}
1,0032e64d-5105-4369-89c4-46f05247c684,Free Company,{'proposals_count': 0}
2,00453db4-685a-4221-9ad1-fda0d82c3a8a,Cream Finance,{'proposals_count': 46}
3,004556b2-f0c0-4843-887c-4b539140ee43,Bleeps DAO,{'proposals_count': 0}
4,0050d8cf-2650-4d66-94dc-77d871b0d755,Psychedelic,{'proposals_count': 0}


In [4]:
df['proposals_count'] = df['propositionStatusMetrics'].map(lambda x: x['proposals_count'])
df.head()

,platform_id,name,propositionStatusMetrics,proposals_count
0,00233765-eb17-4e75-954f-82258bb990aa,Shell Protocol,{'proposals_count': 0},0
1,0032e64d-5105-4369-89c4-46f05247c684,Free Company,{'proposals_count': 0},0
2,00453db4-685a-4221-9ad1-fda0d82c3a8a,Cream Finance,{'proposals_count': 46},46
3,004556b2-f0c0-4843-887c-4b539140ee43,Bleeps DAO,{'proposals_count': 0},0
4,0050d8cf-2650-4d66-94dc-77d871b0d755,Psychedelic,{'proposals_count': 0},0


In [5]:
df['website'] = None
df['additional'] = None
df['platform'] = 'governor'
# df = df[['platform', 'platform_id', 'name', 'website', 'additional', 'proposals_count', 'votes_count']]
df = df[['platform', 'platform_id', 'name', 'website', 'additional', 'proposals_count']]
df.head()

,platform,platform_id,name,website,additional,proposals_count
0,governor,00233765-eb17-4e75-954f-82258bb990aa,Shell Protocol,None,None,0
1,governor,0032e64d-5105-4369-89c4-46f05247c684,Free Company,None,None,0
2,governor,00453db4-685a-4221-9ad1-fda0d82c3a8a,Cream Finance,None,None,46
3,governor,004556b2-f0c0-4843-887c-4b539140ee43,Bleeps DAO,None,None,0
4,governor,0050d8cf-2650-4d66-94dc-77d871b0d755,Psychedelic,None,None,0


In [6]:
# how many have proposals?
df[df['proposals_count'] > 0].shape

(140, 6)

In [7]:
# save to csv
df.to_csv('governor_deployments.csv', index=False)